In [1]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
data = pd.read_csv(r"C:\Users\Harsh\Desktop\GenAI\ANN_ PROJECT\Churn_Modelling.csv")
data.head()

In [4]:
## Preprocess the data  - Drop irrelvant column
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [ ]:
data

In [ ]:
## Encode categorical variable

label_encoder_gender = LabelEncoder()

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
# One Hot Encode Geography Column

from sklearn.preprocessing import OneHotEncoder

one_hot_encoder_geo = OneHotEncoder()

geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder.toarray()

In [ ]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

In [ ]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


In [ ]:
## Combine all the encoded columns with the original data

data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df],axis = 1)
data.head()

In [16]:
## Save the encoder and scaler

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)  

In [ ]:
data.head()

In [18]:
## Divide the dataset into dependent and independent features

X = data.drop('Exited', axis = 1)
y = data['Exited']

## Split the data into training and testing dataset

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state= 42)

## Scale the feature

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [ ]:
X_train

In [19]:
with open('Scaler.pkl','wb') as file:
    pickle.dump(scaler,file)
    

In [ ]:
data

## ANN Implementation

In [16]:
# %pip install tensorflow
# %pip install keras

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime 

In [21]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),  ## Input layer
    Dense(64, activation='relu'),      ## HL 1 Connected with Input layer
    Dense(32, activation='relu'),      ## HL 2 
    Dense(1, activation='sigmoid')     ## Output layer
])

In [ ]:
model.summary()

In [23]:
## Compile the model

model.compile(optimizer= "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [24]:
## Set-up the Tensorboard
import tensorflow

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir , histogram_freq = 1)

In [25]:
## Set-up Early-Stopping

early_stopping_callback= EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights = True)

In [ ]:
## Training model

history = model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs = 100,
                    callbacks = [tensorflow_callback,early_stopping_callback]
                    )

In [ ]:
model.save("model.h5")

In [ ]:
# Load the TensorBoard extension
%load_ext tensorboard

# Start TensorBoard
%tensorboard --logdir logs/fit20240912-224547

In [ ]:
## Load the pickle file

